Krzysztof Pakaszewski

Marek Matys

In [1]:
# Downloading data

!wget https://archive.ics.uci.edu/ml/machine-learning-databases/mechanical-analysis/older-version/mechanical-analysis.data


'wget' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
import pandas as pd

#Creating dataframe
df = pd.read_table("mechanical-analysis.data", delim_whitespace=True, comment= "=", names=["dummy","class","compNumber","sup","cpm","mis","misr","filter","omega"])

#Dropping dummy column because there is irrelevant data
df = df.drop("dummy",axis=1)
df = df.drop(9016) # this row has 0 in support. According to data description that shouldn't happen
print(df.shape)
df.head()

FileNotFoundError: [Errno 2] File mechanical-analysis.data does not exist: 'mechanical-analysis.data'

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
import numpy as np

# 0 - class
# 1 - component number (integer)
# 2 - support in machine (1 - 4)
# 3 - frequency (integer)
# 4 - mis - measure (real)
# 5 - misr - earlier measure (real)
# 6 - filter (labels)
# 7 - omega - rpm (integer, the same for components of one example)

array = df.values

heX = OneHotEncoder()
leY = LabelEncoder()

Y = leY.fit_transform(array[:,0])

Binary = heX.fit_transform(array[:,[2,6]])
X = np.hstack((array[:,[1,3,4,5,7]],Binary.todense()))

print(X.shape)
print(Y.shape)

# in X:
# first 5 columns are numbers
# the rest is binary

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size =0.2, stratify=Y)

scaler = StandardScaler()

X_train[:,:5] = scaler.fit_transform(X_train[:,:5])
X_test[:,:5] = scaler.transform(X_test[:,:5])

print(np.unique(Y,return_counts=True))
print(np.unique(Y_train,return_counts=True))


In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import precision_score, make_scorer
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt

split = StratifiedShuffleSplit(n_splits = 10, test_size=0.2)

precision = make_scorer(precision_score, average='weighted')

params = {'kernel': ['linear','rbf'], 'C': [0.1, 0.5, 1, 5, 10, 50, 100] }
svc = SVC( max_iter=10000)
clf = GridSearchCV(svc, params, scoring=precision, cv=split, verbose =10, n_jobs=-1)

clf.fit(X_train,Y_train)
y_pred = clf.predict(X_test)

print(clf.best_score_)

In [ ]:
from sklearn.metrics import plot_confusion_matrix

precisionScore = precision_score(Y_test, y_pred, average='weighted', zero_division=0)
print("score= ", precisionScore)

plot_confusion_matrix(clf, X_test, Y_test)
plt.show()


